# 直接预测

1. 考虑一条GPS数据预测一个ETA（优先实现）

2. 考虑一段GPS数据预测一个ETA（narrow/wide window）

In [1]:
import os
os.chdir('../')
print(os.path.abspath('.'))
# import pandas as pd
# tar_df = pd.read_csv('data/preprocessed/train_target.csv', header=None, low_memory=False)
# tra_df = pd.read_csv('data/preprocessed/train_trace.csv', header=None, low_memory=False)
# print('target lines: {}'.format(tar_df.shape[0]))
# print('trace lines: {}'.format(tra_df.shape[0]))

e:\Code\ML2020-ETA


## 数据预处理

使用`DataProcessor`类，针对A轮测试数据进行一轮dump


In [2]:
from preprocess import DataProcessor

dataprocessor = DataProcessor(
    output_dir='data/preprocessed'
)

dataprocessor.dump_test_oriented_train_data(
    port_path='data/port.csv',
    # gps_path='data/filtered_data.csv',
    gps_path='data/train0523.csv',
    event_path='data/loadingOrderEvent.csv',
    test_path='data/A_testData0531.csv',
    rewrite=True,
    # debug=True
    debug=False
)

dataprocessor.dump_target(
    port_path='data/port.csv',
    # gps_path='data/filtered_data.csv',
    gps_path='data/train0523.csv',
    event_path='data/loadingOrderEvent.csv',
    test_path='data/A_testData0531.csv',
    rewrite=True,
    # debug=True
    debug=False
)

dataprocessor.dump_test_data(
    port_path='data/port.csv',
    test_data_path='data/A_testData0531.csv',
    rewrite=True,
    # debug=True
    debug=False
)

f.write('Starting to separate data...\n')
dataprocessor.separate_dataset_by_trace(
    source='train',
    trace_map_path='data/preprocessed/train_trace.csv',
    feature_path='data/preprocessed/train_features.csv'
)
dataprocessor.separate_dataset_by_trace(
    source='train',
    trace_map_path='data/preprocessed/train_trace.csv',
    target_path='data/preprocessed/train_target.csv'
)
dataprocessor.separate_dataset_by_trace(
    source='test',
    trace_map_path='data/preprocessed/test_trace.csv',
    feature_path='data/preprocessed/test_features.csv'
)
f.close()
generate validate (train_test_split(x_train, y_train))

## 训练

主要使用lightgbm, xgboost

In [3]:
trace_list = ['CNYTN-MXZLO', 'CNYTN-PAONX', 'CNSHK-CLVAP', 'CNYTN-ARENA', 'CNSHK-MYTPP', 'CNYTN-MATNG', 'CNSHK-GRPIR', 'CNYTN-CAVAN', 'CNHKG-MXZLO', 'CNSHK-SGSIN', 'CNYTN-RTM', 'CNSHA-SGSIN', 'CNSHK-SIKOP', 'COBUN-HKHKG', 'HKHKG-FRFOS', 'CNYTN-NZAKL', 'CNSHK-ESALG', 'CNSHK-ZADUR', 'CNSHA-PAMIT', 'CNSHK-PKQCT', 'CNSHK-LBBEY', 'CNYTN-MTMLA']

from train import train_traces
failed_list = train_traces(trace_list)
success_list = [trace for trace in trace_list if trace not in failed_list]

2: 1.13486e+09	valid_0's l1: 29178.3
Saving model...
x shape 72006
y shape 72006
Starting training...
[1]	valid_0's l2: 8.23116e+10	valid_0's l1: 257108
Training until validation scores don't improve for 20 rounds
[2]	valid_0's l2: 8.23273e+10	valid_0's l1: 257123
[3]	valid_0's l2: 8.23064e+10	valid_0's l1: 257079
[4]	valid_0's l2: 8.22867e+10	valid_0's l1: 257037
[5]	valid_0's l2: 8.22862e+10	valid_0's l1: 257048
[6]	valid_0's l2: 8.22355e+10	valid_0's l1: 256940
[7]	valid_0's l2: 8.22184e+10	valid_0's l1: 256904
[8]	valid_0's l2: 8.21641e+10	valid_0's l1: 256793
[9]	valid_0's l2: 8.217e+10	valid_0's l1: 256774
[10]	valid_0's l2: 8.21357e+10	valid_0's l1: 256719
[11]	valid_0's l2: 8.21662e+10	valid_0's l1: 256777
[12]	valid_0's l2: 8.2178e+10	valid_0's l1: 256779
[13]	valid_0's l2: 8.22e+10	valid_0's l1: 256791
[14]	valid_0's l2: 8.22538e+10	valid_0's l1: 256840
[15]	valid_0's l2: 8.22575e+10	valid_0's l1: 256805
[16]	valid_0's l2: 8.22546e+10	valid_0's l1: 256777
[17]	valid_0's l2: 8

## 测试/预测

先按照A轮的测试数据设计，只有起止港

In [14]:
from predict import pred_traces
import predict
from imp import reload
reload(predict)
preds = predict.pred_traces(success_list)

In [15]:
import pickle
results = {trace: time_deltas for trace, time_deltas in zip(success_list, preds)}
with open('data/results.h5', 'wb') as f:
    pickle.dump(results, f)

In [16]:
print(results)

733.8250179 , 597733.8250179 , 597733.8250179 , 597733.8250179 ,
       597733.8250179 , 597733.8250179 , 597733.8250179 , 597733.8250179 ,
       597733.8250179 , 597733.8250179 , 597733.8250179 , 597733.8250179 ,
       597733.8250179 , 597733.8250179 , 597733.8250179 , 597733.8250179 ,
       597733.8250179 , 597733.8250179 , 597733.8250179 , 597733.8250179 ,
       597733.8250179 , 597733.8250179 , 597733.8250179 , 597733.8250179 ,
       597733.8250179 , 597733.8250179 , 597733.8250179 , 597733.8250179 ,
       597733.8250179 , 597733.8250179 , 597733.8250179 , 597733.8250179 ,
       597733.8250179 , 597733.8250179 , 597733.8250179 , 597733.8250179 ,
       597733.8250179 , 597733.8250179 , 597733.8250179 , 597733.8250179 ,
       597733.8250179 , 597733.8250179 , 597733.8250179 , 597733.8250179 ,
       597733.8250179 , 597733.8250179 , 597733.8250179 , 597733.8250179 ,
       597733.8250179 , 597733.8250179 , 597733.8250179 , 597733.8250179 ]), 'CNSHK-SIKOP': array([1944092., 1